In [21]:
import requests 
from pymongo import MongoClient
from bs4 import BeautifulSoup
import json

# ket noi den mongodb
client = MongoClient('mongodb+srv')  # Thay đổi URL nếu cần
db = client['f1_Database33']  # Tạo hoặc kết nối đến database 'f1_database'
collection = db['drivers']

url = 'https://www.formula1.com/en/results/2024/drivers'
response = requests.get(url)
data = []
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')  # added parser to avoid warnings
    table = soup.find('table', class_='f1-table f1-table-with-data w-full')

    # trich xuat cac dong trong bang
    if table:
        rows = table.find_all('tr')

        for row_id, row in enumerate(rows):
            columns = row.find_all("td")
            if columns:
                position = columns[0].text.strip()
                # lay cac the trong cot driver name
                spans = columns[1].find_all('span')

                if len(spans) >= 2:
                    first_name = spans[0].text.strip()
                    last_name = spans[1].text.strip()
                    driver_name = first_name + " " + last_name

                nationality = columns[2].text.strip()
                car = columns[3].text.strip()
                pts = columns[4].text.strip()

                # Tạo một dictionary cho mỗi hàng
                row_data = {
                    'position': position,
                    'driver_name': driver_name,
                    'nationality': nationality,
                    'car': car,
                    'pts': pts
                }

                # Thêm dictionary vào danh sách
                data.append(row_data)

        # Write data to a JSON file
        with open('F1_.json', 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)
        
        if data:
            collection.insert_many(data)
            print("Data successfully inserted into MongoDB.")
        else:
            print('empty data')
else:
    print('Connection failed')


empty column
Data successfully inserted into MongoDB.
